In [1]:
import os
from xml.dom import minidom
import pandas as pd
import numpy as np
directory = os.getcwd() 
import nltk
import gzip
import gensim 
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
# Extract text from inside

def preprocess(folder):
    no_of_candidates = []
    no_of_paras = []
    current_dir = directory + "/"+folder
    base_cases = []
    entailed_fragments = []
    paragraphs = []
    list_of_files = sorted(os.listdir(current_dir))
    if folder == 'task2_train':
        x=1
    else:
        x = 2
    for i in range(len(list_of_files)-x):
        if i%100 == 0:
            print("Processing ",i)
        f = open(current_dir+'/'+list_of_files[i]+"/base_case.txt","r")
        g = open(current_dir+'/'+list_of_files[i]+"/entailed_fragment.txt","r")
        paragraph_filenames = sorted(os.listdir(current_dir+'/'+list_of_files[i]+"/paragraphs"))
        no_of_candidates.append(len(paragraph_filenames))
        a = f.read()
        b = g.read()
        base_cases.append(a)
        entailed_fragments.append(b)
        f.close()
        g.close()
        
        arr = []
        no_of_paras.append(len(paragraph_filenames))

        for j in range(len(paragraph_filenames)):
            g = open(current_dir+'/'+list_of_files[i]+"/paragraphs/"+paragraph_filenames[j],"r")
            paragraphs.append(g.read())
            g.close()
    return base_cases,entailed_fragments,paragraphs,no_of_paras,no_of_candidates
    
#train_base_cases,train_entailed_fragments, train_paragraphs,train_no_of_paras = preprocess('task2_train')
    
test_base_cases,test_entailed_fragments, test_paragraphs,test_no_of_paras,no_of_candidates = preprocess('task2_test')

Processing  0


In [3]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
import re
from nltk.corpus import stopwords
lemma = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
dict_words = set(nltk.corpus.words.words())
def filt(case_docs):
    filtered_words =[]
    filtered_docs = [ '' for i in range(len(case_docs))]
    lemma_docs = [ '' for i in range(len(case_docs))]
    for i in range(len(case_docs)):
        case_words = re.split("(?:(?:[^a-zA-Z]+')|(?:'[^a-zA-Z]+))|(?:[^a-zA-Z']+)", case_docs[i])
        filtered_word_list = [word.lower() for word in case_words if (( len(word) >= 3 and word.isalpha() and word.lower() not in stop_words ))  ]
        filtered_words.append(filtered_word_list)
        '''or ( len(word) <= 3 and word.isdigit())'''
        for word in filtered_word_list:
            filtered_docs[i] = filtered_docs[i] + word + " "
            lemma_docs[i] = lemma_docs[i] + lemma.lemmatize(word) + " "
    return lemma_docs
test_entailed_fragments = filt(test_entailed_fragments)
test_paragraphs = filt(test_paragraphs)
test_base_cases = filt(test_base_cases)

In [27]:
data = test_entailed_fragments + test_paragraphs
documents = test_entailed_fragments + test_paragraphs + test_base_cases
def gen_labels(data):
    labels = []
    for i in range(len(test_entailed_fragments)):
        labels.append("base_case_"+str(i+1))
    for i in range(len(test_entailed_fragments)):
        for j in range(no_of_candidates[i]):
            labels.append("candidate_"+str(i+1)+"_"+str(j+1))
    for i in range(len(test_entailed_fragments)):
        labels.append("extra_"+str(i+1))
    return labels
labels = gen_labels(documents)

In [28]:
labels[80:130]

['candidate_2_22',
 'candidate_3_1',
 'candidate_3_2',
 'candidate_3_3',
 'candidate_3_4',
 'candidate_3_5',
 'candidate_3_6',
 'candidate_3_7',
 'candidate_3_8',
 'candidate_3_9',
 'candidate_3_10',
 'candidate_3_11',
 'candidate_3_12',
 'candidate_3_13',
 'candidate_3_14',
 'candidate_3_15',
 'candidate_3_16',
 'candidate_3_17',
 'candidate_3_18',
 'candidate_3_19',
 'candidate_3_20',
 'candidate_3_21',
 'candidate_3_22',
 'candidate_3_23',
 'candidate_3_24',
 'candidate_3_25',
 'candidate_3_26',
 'candidate_3_27',
 'candidate_3_28',
 'candidate_3_29',
 'candidate_3_30',
 'candidate_3_31',
 'candidate_3_32',
 'candidate_3_33',
 'candidate_3_34',
 'candidate_3_35',
 'candidate_3_36',
 'candidate_3_37',
 'candidate_3_38',
 'candidate_3_39',
 'candidate_4_1',
 'candidate_4_2',
 'candidate_4_3',
 'candidate_4_4',
 'candidate_4_5',
 'candidate_4_6',
 'candidate_4_7',
 'candidate_4_8',
 'candidate_4_9',
 'candidate_4_10']

In [7]:
import os
from os import listdir
from os.path import isfile, join
from gensim import models
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Word2Vec
from gensim import corpora, models, similarities
import gensim, logging
import time
import re
import nltk
import io
from nltk.corpus import stopwords
import numpy as np

from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
stop_words = set(stopwords.words('english'))
dict_words = set(nltk.corpus.words.words())

In [29]:
class DocIterator(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list

    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield TaggedDocument(words=re.split('\W+',doc), tags=[self.labels_list[idx]])
iterator = DocIterator(documents, labels)

In [30]:
model = gensim.models.Doc2Vec(vector_size=100, window=5, min_count=2, workers=4, alpha=0.025, min_alpha=0.001)
model.build_vocab(iterator)

print('done building vocabulary')
print('start training the model')
tic = time.time()
model.train(iterator,total_examples=model.corpus_count, epochs = 50)
toc = time.time()
print(toc-tic)
model.save("doc2vec_task_2_vec_100_window_5_test_set_minc_2_epoch_50.model")

2019-03-30 19:44:20,320 : INFO : collecting all words and their counts
2019-03-30 19:44:20,321 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-03-30 19:44:20,637 : INFO : collected 14802 word types and 1536 unique tags from a corpus of 1536 examples and 390500 words
2019-03-30 19:44:20,638 : INFO : Loading a fresh vocabulary
2019-03-30 19:44:20,679 : INFO : effective_min_count=2 retains 9829 unique words (66% of original 14802, drops 4973)
2019-03-30 19:44:20,679 : INFO : effective_min_count=2 leaves 385527 word corpus (98% of original 390500, drops 4973)
2019-03-30 19:44:20,740 : INFO : deleting the raw counts dictionary of 14802 items
2019-03-30 19:44:20,742 : INFO : sample=0.001 downsamples 46 most-common words
2019-03-30 19:44:20,744 : INFO : downsampling leaves estimated 293450 word corpus (76.1% of prior 385527)
2019-03-30 19:44:20,799 : INFO : estimated required memory for 9829 words and 100 dimensions: 13699300 bytes
2019-03-30 19:44:20,800 

done building vocabulary
start training the model


2019-03-30 19:44:21,614 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-03-30 19:44:21,615 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-03-30 19:44:21,620 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-03-30 19:44:21,627 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-03-30 19:44:21,628 : INFO : EPOCH - 1 : training on 390500 raw words (294999 effective words) took 0.6s, 455274 effective words/s
2019-03-30 19:44:22,235 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-03-30 19:44:22,236 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-03-30 19:44:22,238 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-03-30 19:44:22,247 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-03-30 19:44:22,248 : INFO : EPOCH - 2 : training on 390500 raw words (295071 effective words) took 0.6s, 482908 effective words/s
20

2019-03-30 19:44:32,189 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-03-30 19:44:32,190 : INFO : EPOCH - 17 : training on 390500 raw words (294939 effective words) took 0.6s, 479725 effective words/s
2019-03-30 19:44:32,786 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-03-30 19:44:32,787 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-03-30 19:44:32,792 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-03-30 19:44:32,798 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-03-30 19:44:32,799 : INFO : EPOCH - 18 : training on 390500 raw words (294808 effective words) took 0.6s, 494916 effective words/s
2019-03-30 19:44:33,399 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-03-30 19:44:33,400 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-03-30 19:44:33,405 : INFO : worker thread finished; awaiting finish of 1 more threads


2019-03-30 19:44:43,200 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-03-30 19:44:43,201 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-03-30 19:44:43,202 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-03-30 19:44:43,210 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-03-30 19:44:43,211 : INFO : EPOCH - 34 : training on 390500 raw words (294819 effective words) took 0.6s, 496582 effective words/s
2019-03-30 19:44:43,839 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-03-30 19:44:43,841 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-03-30 19:44:43,842 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-03-30 19:44:43,851 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-03-30 19:44:43,851 : INFO : EPOCH - 35 : training on 390500 raw words (294967 effective words) took 0.6s, 472651 effective words/s


2019-03-30 19:44:53,631 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-03-30 19:44:53,632 : INFO : EPOCH - 50 : training on 390500 raw words (295196 effective words) took 0.6s, 490804 effective words/s
2019-03-30 19:44:53,633 : INFO : training on a 19525000 raw words (14750903 effective words) took 32.7s, 451649 effective words/s
2019-03-30 19:44:53,634 : INFO : saving Doc2Vec object under doc2vec_task_2_vec_100_window_5_test_set_minc_2_epoch_50.model, separately None
2019-03-30 19:44:53,789 : INFO : saved doc2vec_task_2_vec_100_window_5_test_set_minc_2_epoch_50.model


32.66176462173462


In [ ]:
from gensim.models.doc2vec import Doc2Vec
model = Doc2Vec.load("doc2vec_task_1_vec_150_window_10_test_set_minc_2_epoch_50.model")

In [32]:
model.docvecs.most_similar(labels[0],topn=50)

[('candidate_4_1', 0.756076455116272),
 ('candidate_17_1', 0.7485694885253906),
 ('candidate_26_22', 0.7384686470031738),
 ('candidate_36_19', 0.7326580286026001),
 ('base_case_4', 0.7252758741378784),
 ('candidate_14_14', 0.7201084494590759),
 ('candidate_29_11', 0.7182266116142273),
 ('candidate_25_43', 0.7154099941253662),
 ('candidate_19_23', 0.7154039144515991),
 ('candidate_44_27', 0.7145591974258423),
 ('base_case_44', 0.7120709419250488),
 ('candidate_21_65', 0.711906909942627),
 ('candidate_25_46', 0.7117351293563843),
 ('candidate_38_25', 0.7116952538490295),
 ('candidate_28_45', 0.7114608287811279),
 ('base_case_31', 0.7110193967819214),
 ('candidate_3_23', 0.7100916504859924),
 ('candidate_24_39', 0.7073547840118408),
 ('candidate_1_8', 0.7066588997840881),
 ('base_case_28', 0.7065763473510742),
 ('candidate_37_2', 0.7058336734771729),
 ('candidate_32_30', 0.7055041790008545),
 ('base_case_3', 0.7045915126800537),
 ('candidate_3_39', 0.7038944363594055),
 ('candidate_36_32'

In [36]:
for i in range(44):
    #print("\nPredicting  For ",i)
    lst = []
    for j in range(no_of_candidates[i]):
        #lst.append((model.docvecs.similarity(labels[i],labels[285+i*200+j]),j+1))
        lst.append((model.docvecs.similarity(labels[i],"candidate_"+str(i+1)+"_"+str(j+1)),j+1))
    lst.sort(reverse=True)
    cutoff = ((lst[0][0] + lst[1][0])/2)*0.90
    for j in range(1):
        if lst[j][0] > cutoff:
            print("t2-"+str(i+1),lst[j][1],"IITP2d2v")

t2-1 8 IITP2d2v
t2-2 4 IITP2d2v
t2-3 39 IITP2d2v
t2-4 20 IITP2d2v
t2-5 61 IITP2d2v
t2-6 30 IITP2d2v
t2-7 25 IITP2d2v
t2-8 26 IITP2d2v
t2-9 15 IITP2d2v
t2-10 4 IITP2d2v
t2-11 1 IITP2d2v
t2-12 24 IITP2d2v
t2-13 5 IITP2d2v
t2-14 14 IITP2d2v
t2-15 19 IITP2d2v
t2-16 59 IITP2d2v
t2-17 15 IITP2d2v
t2-18 18 IITP2d2v
t2-19 23 IITP2d2v
t2-20 28 IITP2d2v
t2-21 62 IITP2d2v
t2-22 21 IITP2d2v
t2-23 17 IITP2d2v
t2-24 20 IITP2d2v
t2-25 43 IITP2d2v
t2-26 22 IITP2d2v
t2-27 18 IITP2d2v
t2-28 45 IITP2d2v
t2-29 11 IITP2d2v
t2-30 31 IITP2d2v
t2-31 31 IITP2d2v
t2-32 30 IITP2d2v
t2-33 9 IITP2d2v
t2-34 4 IITP2d2v
t2-35 7 IITP2d2v
t2-36 1 IITP2d2v
t2-37 22 IITP2d2v
t2-38 25 IITP2d2v
t2-39 12 IITP2d2v
t2-40 32 IITP2d2v
t2-41 15 IITP2d2v
t2-42 1 IITP2d2v
t2-43 6 IITP2d2v
t2-44 27 IITP2d2v


In [ ]:
correct_pred

In [ ]:
total_true

In [ ]:
total_pred

In [ ]:
325,1486,855

In [ ]:
588/2421

In [ ]:
468/1486

In [ ]:
import re
a = re.search("Baban","I am Baban Gain Baban GAin")
print(a)